In [158]:
import numpy as np
import collections.abc
collections.Iterable = collections.abc.Iterable
from collections import OrderedDict
import pylogit as pl
import pandas as pd
import scipy.special
import json

In [159]:
df = pd.read_csv('history.csv')
df

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
0,3574,0,7,19,4,13,0,0,0,0,...,0,5,2,3,9,6,2,3,11,3
1,3574,1,18,12,17,14,0,0,0,0,...,0,13,2,3,3,13,2,3,3,4
2,3574,2,2,16,9,3,0,0,0,0,...,0,13,2,5,4,13,2,6,5,2
3,3574,3,10,20,1,6,0,0,0,0,...,0,13,10,2,13,13,13,2,13,3
4,3574,4,8,15,11,5,0,0,0,0,...,0,13,2,11,6,13,2,13,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16840,8641,0,19,12,2,15,2,4,3,3,...,-1,4,13,13,2,4,13,13,2,4
16841,8641,1,10,20,8,4,1,-1,-2,3,...,0,13,8,13,2,13,12,13,2,4
16842,8641,2,17,7,18,6,4,-1,2,1,...,-1,2,6,5,4,2,6,5,4,1
16843,8641,3,16,1,5,14,-2,-1,4,1,...,-1,13,4,2,5,13,5,2,7,2


In [160]:
df.describe()

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
count,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,...,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000,16845.000000
mean,6829.771149,2.000000,10.591036,10.431345,10.531374,10.446245,0.970377,0.985634,0.964797,0.985159,...,-1.208549,7.915702,7.034194,5.982191,4.689225,8.237934,7.407183,6.360641,5.058949,2.744672
std,1089.898648,1.414256,5.751749,5.783485,5.783417,5.745589,1.731506,1.735176,1.731504,1.731130,...,0.970179,4.460321,4.417173,4.182325,3.636474,4.533579,4.551443,4.382981,3.927148,1.106017
min,3574.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-5.000000,-5.000000,-5.000000,-5.000000,...,-6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
25%,5932.000000,1.000000,6.000000,5.000000,6.000000,6.000000,0.000000,0.000000,0.000000,0.000000,...,-2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000
50%,6857.000000,2.000000,11.000000,10.000000,11.000000,10.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,7.000000,6.000000,4.000000,3.000000,9.000000,7.000000,5.000000,3.000000,3.000000
75%,7799.000000,3.000000,16.000000,15.000000,16.000000,15.000000,2.000000,2.000000,2.000000,2.000000,...,0.000000,13.000000,13.000000,10.000000,6.000000,13.000000,13.000000,11.000000,7.000000,4.000000
max,8641.000000,4.000000,20.000000,20.000000,20.000000,20.000000,8.000000,8.000000,8.000000,9.000000,...,0.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,4.000000


In [161]:
# combine columns pirate1 through pirate4 into a single column containing a list of pirate choices
def convert_df(df_orig):
    df = df_orig.copy()
    df['pirates'] = df[['pirate1', 'pirate2', 'pirate3', 'pirate4']].values.tolist()
    df['fas'] = df[['fa1', 'fa2', 'fa3', 'fa4']].values.tolist()
    df['pfas'] = df[['pfa1', 'pfa2', 'pfa3', 'pfa4']].values.tolist()
    df['nfas'] = df[['nfa1', 'nfa2', 'nfa3', 'nfa4']].values.tolist()
    df['opening_odds'] = df[['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4']].values.tolist()
    df['closing_odds'] = df[['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']].values.tolist()
    df = df.drop(['pirate1', 'pirate2', 'pirate3', 'pirate4'], axis=1)
    df = df.drop(['fa1', 'fa2', 'fa3', 'fa4'], axis=1)
    df = df.drop(['pfa1', 'pfa2', 'pfa3', 'pfa4'], axis=1)
    df = df.drop(['nfa1', 'nfa2', 'nfa3', 'nfa4'], axis=1)
    df = df.drop(['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4'], axis=1)
    df = df.drop(['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4'], axis=1)
    df['match_id'] = df['round'] * 5 + df['arena']
    return df
df = convert_df(df)
df

,round,arena,winner,pirates,fas,pfas,nfas,opening_odds,closing_odds,match_id
0,3574,0,3,"[7, 19, 4, 13]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[5, 2, 3, 9]","[6, 2, 3, 11]",17870
1,3574,1,4,"[18, 12, 17, 14]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 3, 3]","[13, 2, 3, 3]",17871
2,3574,2,2,"[2, 16, 9, 3]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 5, 4]","[13, 2, 6, 5]",17872
3,3574,3,3,"[10, 20, 1, 6]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 10, 2, 13]","[13, 13, 2, 13]",17873
4,3574,4,2,"[8, 15, 11, 5]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 11, 6]","[13, 2, 13, 6]",17874
...,...,...,...,...,...,...,...,...,...,...
16840,8641,0,4,"[19, 12, 2, 15]","[2, 4, 3, 3]","[2, 5, 5, 4]","[0, -1, -2, -1]","[4, 13, 13, 2]","[4, 13, 13, 2]",43205
16841,8641,1,4,"[10, 20, 8, 4]","[1, -1, -2, 3]","[1, 1, 1, 3]","[0, -2, -3, 0]","[13, 8, 13, 2]","[13, 12, 13, 2]",43206
16842,8641,2,1,"[17, 7, 18, 6]","[4, -1, 2, 1]","[5, 1, 4, 2]","[-1, -2, -2, -1]","[2, 6, 5, 4]","[2, 6, 5, 4]",43207
16843,8641,3,2,"[16, 1, 5, 14]","[-2, -1, 4, 1]","[0, 1, 4, 2]","[-2, -2, 0, -1]","[13, 4, 2, 5]","[13, 5, 2, 7]",43208


In [163]:
# explode each row of df into four rows, so that the resulting df
# contains columns of round, arena, win (0 or 1), pirate, fa, opening_odds, closing_odds, match_id
def convert_to_long_format_with_win(df):
    long_format_columns = ['round', 'arena', 'win', 'pirate', 'fa', 'pfa', 'nfa', 'position', 'is_pos1', 'is_pos2', 'is_pos3', 'is_pos4', 'opening_odds', 'closing_odds', 'match_id']
    # long_format = pd.DataFrame([], columns=long_format_columns)
    long_format = pd.DataFrame([
        (
            tup.round,
            tup.arena,
            1 if 'winner' in df.columns and tup.winner == (i + 1) else 0,
            tup.pirates[i],
            tup.fas[i],
            tup.pfas[i],
            tup.nfas[i],
            i + 1,
            1 if i + 1 == 1 else 0,
            1 if i + 1 == 2 else 0,
            1 if i + 1 == 3 else 0,
            1 if i + 1 == 4 else 0,
            tup.opening_odds[i],
            tup.closing_odds[i],
            tup.match_id
        )
        for tup in df.itertuples() for i in range(4)
    ],columns=long_format_columns)
    return long_format

def convert_to_long_format_without_win(df):
    return convert_to_long_format_with_win(df).drop(['win'], axis=1)

long_format = convert_to_long_format_with_win(df)
long_format

,round,arena,win,pirate,fa,pfa,nfa,position,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id
0,3574,0,0,7,0,0,0,1,1,0,0,0,5,6,17870
1,3574,0,0,19,0,0,0,2,0,1,0,0,2,2,17870
2,3574,0,1,4,0,0,0,3,0,0,1,0,3,3,17870
3,3574,0,0,13,0,0,0,4,0,0,0,1,9,11,17870
4,3574,1,0,18,0,0,0,1,1,0,0,0,13,13,17871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67375,8641,3,0,14,1,2,-1,4,0,0,0,1,5,7,43208
67376,8641,4,0,11,0,1,-1,1,1,0,0,0,11,8,43209
67377,8641,4,1,13,-1,1,-2,2,0,1,0,0,3,4,43209
67378,8641,4,0,3,3,4,-1,3,0,0,1,0,3,3,43209


In [140]:
long_format['log_opening_implied_winrate'] = np.log(1.0 / long_format['opening_odds'])
long_format

,round,arena,win,pirate,fa,pfa,nfa,position,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,3574,0,0,7,0,0,0,1,5,6,17870,-1.609438
1,3574,0,0,19,0,0,0,2,2,2,17870,-0.693147
2,3574,0,1,4,0,0,0,3,3,3,17870,-1.098612
3,3574,0,0,13,0,0,0,4,9,11,17870,-2.197225
4,3574,1,0,18,0,0,0,1,13,13,17871,-2.564949
...,...,...,...,...,...,...,...,...,...,...,...,...
67375,8641,3,0,14,1,2,-1,4,5,7,43208,-1.609438
67376,8641,4,0,11,0,1,-1,1,11,8,43209,-2.397895
67377,8641,4,1,13,-1,1,-2,2,3,4,43209,-1.098612
67378,8641,4,0,3,3,4,-1,3,3,3,43209,-1.098612


In [141]:
PIRATE_NAMES = {
    1: "Dan",
    2: "Sproggie",
    3: "Orvinn",
    4: "Lucky",
    5: "Edmund",
    6: "Peg Leg",
    7: "Bonnie",
    8: "Puffo",
    9: "Stuff",
    10: "Squire",
    11: "Crossblades",
    12: "Stripey",
    13: "Ned",
    14: "Fairfax",
    15: "Gooblah",
    16: "Franchisco",
    17: "Federismo",
    18: "Blackbeard",
    19: "Buck",
    20: "Tailhook",
}

In [60]:
pirates_including_the_first = list(range(1, 21))
pirates_except_the_first = list(range(2, 21))

In [61]:
with_loiw_specification = OrderedDict()
with_loiw_names = OrderedDict()
with_loiw_specification["intercept"] = pirates_except_the_first
with_loiw_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
with_loiw_specification["fa"] = [pirates_including_the_first]
with_loiw_names["fa"] = ['FA']
with_loiw_specification["log_opening_implied_winrate"] = [pirates_including_the_first]
with_loiw_names["log_opening_implied_winrate"] = ['log_opening_implied_winrate']
with_loiw_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=with_loiw_specification,
model_type='MNL',
names=with_loiw_names)
with_loiw_mnl.fit_mle(np.zeros(21))
with_loiw_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.32 seconds.
Final log-likelihood: -18,235.0666


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,824
Method:                                MLE   Df Model:                           21
Date:                     Wed, 04 Jan 2023   Pseudo R-squ.:                   0.219
Time:                             01:16:12   Pseudo R-bar-squ.:               0.218
AIC:                            36,512.133   Log-Likelihood:            -18,235.067
BIC:                            36,674.501   LL-Null:                   -23,352.129
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
ASC_2_Sproggie                 -0.3015      0.064     -4.747      0.000      -0.426      -0.177
ASC_3_Orvinn                   -0.6093      0.082     -7.432      0.000      -0.770      -0.449
ASC_4_Lucky                    -0.1268      0.055     -2.319      0.020      -0.234      -0.020
ASC_5_Edmund                   -0.1442      0.055     -2.600      0.009      -0.253      -0.036
ASC_6_Peg Leg                  -0.3446      0.068     -5.051      0.000      -0.478      -0.211
ASC_7_Bonnie                   -0.3013      0.059     -5.084      0.000      -0.418      -0.185
ASC_8_Puffo                    -0.4196      0.074     -5.696      0.000      -0.564      -0.275
ASC_9_Stuff                    -0.7360      0.094     -7.799      0.000      -0.921      -0.551
ASC_10_Squire                  -0.6544      0.090     -7.307      0.000      -0.830      -0.479
ASC_11_Crossblades             -0.5264      0.079     -6.680      0.000      -0.681      -0.372
ASC_12_Stripey                 -0.2692      0.062     -4.333      0.000      -0.391      -0.147
ASC_13_Ned                     -0.1619      0.058     -2.806      0.005      -0.275      -0.049
ASC_14_Fairfax                 -0.4022      0.065     -6.163      0.000      -0.530      -0.274
ASC_15_Gooblah                  0.4210      0.053      7.962      0.000       0.317       0.525
ASC_16_Franchisco              -0.1499      0.052     -2.875      0.004      -0.252      -0.048
ASC_17_Federismo               -0.1096      0.052     -2.103      0.035      -0.212      -0.007
ASC_18_Blackbeard              -0.3519      0.066     -5.325      0.000      -0.481      -0.222
ASC_19_Buck                     0.2487      0.052      4.751      0.000       0.146       0.351
ASC_20_Tailhook                -0.0842      0.053     -1.584      0.113      -0.188       0.020
FA                              0.0951      0.009     11.029      0.000       0.078       0.112
log_opening_implied_winrate     0.8761      0.028     31.458      0.000       0.821       0.931
===============================================================================================
"""

In [62]:
# train a even more sophisticated model that includes the position of the pirate in the lineup
with_loiw_and_position_specification = OrderedDict()
with_loiw_and_position_names = OrderedDict()
with_loiw_and_position_specification["intercept"] = pirates_except_the_first
with_loiw_and_position_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
with_loiw_and_position_specification["fa"] = [pirates_including_the_first]
with_loiw_and_position_names["fa"] = ['FA']
with_loiw_and_position_specification["log_opening_implied_winrate"] = [pirates_including_the_first]
with_loiw_and_position_names["log_opening_implied_winrate"] = ['log_opening_implied_winrate']
with_loiw_and_position_specification["position"] = [pirates_including_the_first]
with_loiw_and_position_names["position"] = ['position']
with_loiw_and_position_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=with_loiw_and_position_specification,
model_type='MNL',
names=with_loiw_and_position_names)
with_loiw_and_position_mnl.fit_mle(np.zeros(22))
with_loiw_and_position_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.32 seconds.
Final log-likelihood: -18,145.0088


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,823
Method:                                MLE   Df Model:                           22
Date:                     Wed, 04 Jan 2023   Pseudo R-squ.:                   0.223
Time:                             01:16:18   Pseudo R-bar-squ.:               0.222
AIC:                            36,334.018   Log-Likelihood:            -18,145.009
BIC:                            36,504.117   LL-Null:                   -23,352.129
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
ASC_2_Sproggie                 -0.6864      0.070     -9.779      0.000      -0.824      -0.549
ASC_3_Orvinn                   -1.2715      0.097    -13.109      0.000      -1.462      -1.081
ASC_4_Lucky                    -0.2781      0.056     -4.950      0.000      -0.388      -0.168
ASC_5_Edmund                   -0.3329      0.058     -5.780      0.000      -0.446      -0.220
ASC_6_Peg Leg                  -0.7212      0.074     -9.726      0.000      -0.866      -0.576
ASC_7_Bonnie                   -0.5968      0.064     -9.373      0.000      -0.722      -0.472
ASC_8_Puffo                    -0.9639      0.085    -11.317      0.000      -1.131      -0.797
ASC_9_Stuff                    -1.3771      0.107    -12.833      0.000      -1.587      -1.167
ASC_10_Squire                  -1.2387      0.101    -12.295      0.000      -1.436      -1.041
ASC_11_Crossblades             -1.1123      0.091    -12.208      0.000      -1.291      -0.934
ASC_12_Stripey                 -0.6555      0.069     -9.495      0.000      -0.791      -0.520
ASC_13_Ned                     -0.3668      0.060     -6.117      0.000      -0.484      -0.249
ASC_14_Fairfax                 -0.8174      0.073    -11.221      0.000      -0.960      -0.675
ASC_15_Gooblah                  0.6573      0.056     11.635      0.000       0.547       0.768
ASC_16_Franchisco              -0.2924      0.054     -5.449      0.000      -0.398      -0.187
ASC_17_Federismo               -0.2564      0.054     -4.768      0.000      -0.362      -0.151
ASC_18_Blackbeard              -0.7572      0.073    -10.331      0.000      -0.901      -0.614
ASC_19_Buck                     0.3713      0.054      6.919      0.000       0.266       0.477
ASC_20_Tailhook                -0.1907      0.054     -3.526      0.000      -0.297      -0.085
FA                              0.1730      0.011     16.307      0.000       0.152       0.194
log_opening_implied_winrate     0.5182      0.039     13.415      0.000       0.442       0.594
position                        0.1458      0.011     13.288      0.000       0.124       0.167
===============================================================================================
"""

In [63]:
# train a reduced model without log opening implied winrate
# i.e. we assume that the opening odds don't tell us any extra information
simple_specification = OrderedDict()
simple_names = OrderedDict()
simple_specification["intercept"] = pirates_except_the_first
simple_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
simple_specification["fa"] = [pirates_including_the_first]
simple_names["fa"] = ['FA']

simple_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=simple_specification,
model_type='MNL',
names=simple_names)
simple_mnl.fit_mle(np.zeros(20))
simple_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.52 seconds.
Final log-likelihood: -18,750.3869


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,825
Method:                                MLE   Df Model:                           20
Date:                     Wed, 04 Jan 2023   Pseudo R-squ.:                   0.197
Time:                             01:16:22   Pseudo R-bar-squ.:               0.196
AIC:                            37,540.774   Log-Likelihood:            -18,750.387
BIC:                            37,695.410   LL-Null:                   -23,352.129
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.1955      0.057    -21.044      0.000      -1.307      -1.084
ASC_3_Orvinn          -2.1749      0.067    -32.570      0.000      -2.306      -2.044
ASC_4_Lucky           -0.4858      0.053     -9.130      0.000      -0.590      -0.381
ASC_5_Edmund          -0.5922      0.053    -11.110      0.000      -0.697      -0.488
ASC_6_Peg Leg         -1.2232      0.062    -19.679      0.000      -1.345      -1.101
ASC_7_Bonnie          -0.9844      0.055    -17.846      0.000      -1.092      -0.876
ASC_8_Puffo           -1.6893      0.063    -26.957      0.000      -1.812      -1.567
ASC_9_Stuff           -2.2721      0.082    -27.691      0.000      -2.433      -2.111
ASC_10_Squire         -2.0243      0.079    -25.583      0.000      -2.179      -1.869
ASC_11_Crossblades    -1.8984      0.067    -28.489      0.000      -2.029      -1.768
ASC_12_Stripey        -1.1772      0.055    -21.348      0.000      -1.285      -1.069
ASC_13_Ned            -0.6512      0.055    -11.808      0.000      -0.759      -0.543
ASC_14_Fairfax        -1.3846      0.058    -23.989      0.000      -1.498      -1.271
ASC_15_Gooblah         0.9588      0.051     18.983      0.000       0.860       1.058
ASC_16_Franchisco     -0.4885      0.051     -9.593      0.000      -0.588      -0.389
ASC_17_Federismo      -0.4654      0.051     -9.112      0.000      -0.565      -0.365
ASC_18_Blackbeard     -1.3032      0.059    -22.103      0.000      -1.419      -1.188
ASC_19_Buck            0.5275      0.052     10.212      0.000       0.426       0.629
ASC_20_Tailhook       -0.3319      0.052     -6.349      0.000      -0.434      -0.229
FA                     0.2798      0.007     42.317      0.000       0.267       0.293
======================================================================================
"""

In [64]:
# train a reduced model without log opening implied winrate but includes position
# i.e. we assume that the different positions are rigged, the opening odds don't tell us any extra information
with_position_specification = OrderedDict()
with_position_names = OrderedDict()
with_position_specification["intercept"] = pirates_except_the_first
with_position_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
with_position_specification["fa"] = [pirates_including_the_first]
with_position_names["fa"] = ['FA']
with_position_specification["position"] = [pirates_including_the_first]
with_position_names["position"] = ['position']

with_position_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=with_position_specification,
model_type='MNL',
names=with_position_names)
with_position_mnl.fit_mle(np.zeros(21))
with_position_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.34 seconds.
Final log-likelihood: -18,235.2126


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,824
Method:                                MLE   Df Model:                           21
Date:                     Wed, 04 Jan 2023   Pseudo R-squ.:                   0.219
Time:                             01:16:27   Pseudo R-bar-squ.:               0.218
AIC:                            36,512.425   Log-Likelihood:            -18,235.213
BIC:                            36,674.793   LL-Null:                   -23,352.129
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.2326      0.058    -21.391      0.000      -1.346      -1.120
ASC_3_Orvinn          -2.2261      0.068    -32.912      0.000      -2.359      -2.094
ASC_4_Lucky           -0.4951      0.054     -9.166      0.000      -0.601      -0.389
ASC_5_Edmund          -0.6035      0.054    -11.142      0.000      -0.710      -0.497
ASC_6_Peg Leg         -1.2516      0.063    -19.879      0.000      -1.375      -1.128
ASC_7_Bonnie          -1.0144      0.056    -18.140      0.000      -1.124      -0.905
ASC_8_Puffo           -1.7435      0.063    -27.459      0.000      -1.868      -1.619
ASC_9_Stuff           -2.3205      0.083    -28.039      0.000      -2.483      -2.158
ASC_10_Squire         -2.0831      0.080    -26.068      0.000      -2.240      -1.926
ASC_11_Crossblades    -1.9504      0.068    -28.892      0.000      -2.083      -1.818
ASC_12_Stripey        -1.2068      0.056    -21.546      0.000      -1.317      -1.097
ASC_13_Ned            -0.6591      0.056    -11.775      0.000      -0.769      -0.549
ASC_14_Fairfax        -1.4113      0.058    -24.126      0.000      -1.526      -1.297
ASC_15_Gooblah         0.9961      0.051     19.411      0.000       0.895       1.097
ASC_16_Franchisco     -0.4977      0.052     -9.613      0.000      -0.599      -0.396
ASC_17_Federismo      -0.4702      0.052     -9.064      0.000      -0.572      -0.368
ASC_18_Blackbeard     -1.3384      0.060    -22.384      0.000      -1.456      -1.221
ASC_19_Buck            0.5466      0.053     10.406      0.000       0.444       0.650
ASC_20_Tailhook       -0.3407      0.053     -6.415      0.000      -0.445      -0.237
FA                     0.2864      0.007     42.667      0.000       0.273       0.300
position               0.2504      0.008     31.589      0.000       0.235       0.266
======================================================================================
"""

In [51]:
POSITIVE_FAS = {
    1: {1: 2, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 2, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 2, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 1, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    4: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    8: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 1, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    12: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    13: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    14: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    16: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 2, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 2, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 2},
    18: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    19: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0}
}
NEGATIVE_FAS = {
    1: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    4: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    8: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    12: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    13: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    14: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    16: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    18: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    19: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0}
}

def get_ongoing_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [69]:
# example of prediction
round_8642 = convert_to_long_format_without_win(convert_df(get_ongoing_df_from_file('raw_json/8642.json')))
prediction_array = with_position_mnl.predict(round_8642)
round_8642

,round,arena,pirate,fa,position,opening_odds,closing_odds,match_id,intercept
0,8642,0,18,0,1,7,6,43210,1.0
1,8642,0,9,0,2,13,13,43210,1.0
2,8642,0,20,0,3,2,2,43210,1.0
3,8642,0,7,0,4,2,2,43210,1.0
4,8642,1,1,6,1,4,6,43211,1.0
5,8642,1,15,5,2,2,2,43211,1.0
6,8642,1,14,1,3,13,13,43211,1.0
7,8642,1,19,0,4,10,13,43211,1.0
8,8642,2,6,2,1,5,5,43212,1.0
9,8642,2,4,1,2,2,2,43212,1.0


In [70]:
prediction_array

array([0.1125368 , 0.05413828, 0.50355994, 0.32976498, 0.22911598,
       0.59841925, 0.0220138 , 0.15045096, 0.1723493 , 0.35425897,
       0.28983166, 0.18356008, 0.02994211, 0.19297443, 0.28019508,
       0.49688838, 0.16284138, 0.16606903, 0.13098133, 0.54010827])

In [78]:
# backtesting
def get_data_for_round(df, round):
    return df[df['round'] == round]

convert_to_long_format_with_win(get_data_for_round(df, 8641))


,round,arena,win,pirate,fa,position,opening_odds,closing_odds,match_id
0,8641,0,0,19,2,1,4,4,43205
1,8641,0,0,12,4,2,13,13,43205
2,8641,0,0,2,3,3,13,13,43205
3,8641,0,1,15,3,4,2,2,43205
4,8641,1,0,10,1,1,13,13,43206
5,8641,1,0,20,-1,2,8,12,43206
6,8641,1,0,8,-2,3,13,13,43206
7,8641,1,1,4,3,4,2,2,43206
8,8641,2,1,17,4,1,2,2,43207
9,8641,2,0,7,-1,2,6,6,43207


In [81]:
def generate_bets():
    for i in range(5):
        for j in range(5):
            for k in range(5):
                for l in range(5):
                    for m in range(5):
                        if i != 0 or j != 0 or k != 0 or l != 0 or m != 0:
                            yield (i, j, k, l, m)

len(list(generate_bets()))

3124

In [101]:
def get_odds_of_bet(bet, odds):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= odds[arena * 4 + pos - 1]
    return prod

tmp = convert_to_long_format_with_win(get_data_for_round(df, 8641))
get_odds_of_bet((4,2,0,0,2), tmp['closing_odds'])

96

In [87]:
def get_estimated_probabilities(df, model):
    return model.predict(df)

get_estimated_probabilities(tmp, with_position_mnl)

array([0.16157463, 0.06373219, 0.05991459, 0.71477859, 0.04079771,
       0.16877705, 0.04003437, 0.75039087, 0.50519039, 0.08993386,
       0.19730162, 0.20757413, 0.07093716, 0.19958858, 0.58709458,
       0.14237969, 0.10620913, 0.3726731 , 0.31408067, 0.2070371 ])

In [102]:
def get_estimated_probability_of_bet(bet, probabilities):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= probabilities[arena * 4 + pos - 1]
    return prod

get_estimated_probability_of_bet((4,2,0,0,2), get_estimated_probabilities(tmp, with_position_mnl))

0.04495861984084361

In [103]:
def get_estimated_return_per_unit_of_bet(bet, odds, probabilities):
    return get_odds_of_bet(bet, odds) * get_estimated_probability_of_bet(bet, probabilities)

get_estimated_return_per_unit_of_bet((4,2,0,0,2), tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl))

4.316027504720987

In [104]:
def get_max_possible_bet_amount_floor(odds):
    return 1000000 // odds

def get_max_possible_bet_amount_ceil(odds):
    return -(1000000 // -odds)

get_max_possible_bet_amount_floor(96)
get_max_possible_bet_amount_ceil(96)

10417

In [105]:
def get_estimated_return_of_bet_amount(bet, odds, probabilities, bet_amount):
    odds_of_bet = get_odds_of_bet(bet, odds)
    estimated_probability = get_estimated_probability_of_bet(bet, probabilities)
    payout = min(1000000, bet_amount * odds_of_bet)
    return payout * estimated_probability

def get_best_bet_amount_and_estimated_return(bet, odds, probabilities):
    max_possible_bet_amount_floor = get_max_possible_bet_amount_floor(get_odds_of_bet(bet, odds))
    max_possible_bet_amount_ceil = get_max_possible_bet_amount_ceil(get_odds_of_bet(bet, odds))
    estimated_return_bet_amount_floor = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_floor)
    estimated_return_bet_amount_ceil = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_ceil)
    if (estimated_return_bet_amount_floor - max_possible_bet_amount_floor) >= (estimated_return_bet_amount_ceil - max_possible_bet_amount_ceil):
        return max_possible_bet_amount_floor, estimated_return_bet_amount_floor
    else:
        return max_possible_bet_amount_ceil, estimated_return_bet_amount_ceil

def get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap):
    amount, estimated_return = get_best_bet_amount_and_estimated_return(bet, odds, probabilities)
    if cap >= amount:
        return amount, estimated_return
    else:
        return cap, get_estimated_return_of_bet_amount(bet, odds, probabilities, cap)


(10417, 44958.61984084361)

In [106]:

get_best_bet_amount_and_estimated_return_given_personal_cap((4,2,0,0,2), tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl), 11376)

(10417, 44958.61984084361)

In [107]:
get_best_bet_amount_and_estimated_return_given_personal_cap((4,4,1,3,2), tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl), 11376)

(11376, 43163.815662257344)

In [110]:
def get_profitable_bets(odds, probabilities, cap):
    for bet in generate_bets():
        if get_estimated_return_per_unit_of_bet(bet, odds, probabilities) > 1:
            bet_amount, estimated_return = get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap)
            estimated_profit = estimated_return - bet_amount
            return_if_won = min(1000000, bet_amount * get_odds_of_bet(bet, odds))
            yield bet, bet_amount, estimated_return, estimated_profit, return_if_won

# 10 best bets for round 8641
sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl), 11376)), key=lambda x: x[3], reverse=True)[:10]

[((4, 2, 0, 0, 2), 10417, 44958.61984084361, 34541.61984084361, 1000000),
 ((4, 4, 1, 3, 2), 11376, 43163.815662257344, 31787.815662257344, 728064),
 ((4, 4, 0, 3, 2), 11376, 42720.34501306255, 31344.34501306255, 364032),
 ((4, 2, 0, 3, 0), 11376, 38674.42059593612, 27298.42059593612, 546048),
 ((0, 2, 0, 3, 2), 10417, 36927.465501978426, 26510.465501978426, 1000000),
 ((4, 4, 1, 0, 2), 11376, 36760.53007304534, 25384.53007304534, 364032),
 ((4, 2, 1, 3, 0), 10417, 35780.63776446788, 25363.63776446788, 1000000),
 ((4, 4, 0, 0, 2), 11376, 36382.84761179591, 25006.847611795907, 182016),
 ((4, 4, 0, 2, 2), 11376, 36308.003888469844, 24932.003888469844, 910080),
 ((4, 4, 3, 0, 2), 11376, 35891.97356129807, 24515.973561298073, 910080)]

In [111]:
def is_winning_bet(bet, winners):
    for arena, pos in enumerate(bet):
        if pos != 0 and winners[arena] != pos:
            return False
    return True

def get_actual_return_of_bet(bet, winners, return_if_won):
    if is_winning_bet(bet, winners):
        return min(1000000, return_if_won)
    else:
        return 0


In [124]:
history_rounds = df["round"].unique().tolist()
len(history_rounds)

3369

In [128]:
from tqdm import tqdm
def backtest_model_with_cap(mnl, cap, history_rounds):
    total_bet_amount = 0
    total_return = 0
    total_potential_bet_amount = cap * 10 * len(history_rounds)
    for round in tqdm(history_rounds):
        tmp = convert_to_long_format_with_win(get_data_for_round(df, round))
        winners = df[df['round'] == round]['winner'].tolist()
        best_ten_bets = sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, mnl), cap)), key=lambda x: x[3], reverse=True)[:10]
        for bet, bet_amount, estimated_return, estimated_profit, return_if_won in best_ten_bets:
            total_bet_amount += bet_amount
            total_return += get_actual_return_of_bet(bet, winners, return_if_won)
    return total_bet_amount, total_potential_bet_amount, total_return, total_return - total_bet_amount, 1 + (total_return - total_bet_amount) / total_potential_bet_amount

In [130]:
backtest_model_with_cap(with_position_mnl, 10000, history_rounds)

100%|██████████| 3369/3369 [03:11<00:00, 17.63it/s]


(325253969, 336900000, 647768680, 322514711, 1.9573010121697834)

In [131]:
backtest_model_with_cap(with_position_mnl, 1000, history_rounds)

100%|██████████| 3369/3369 [03:20<00:00, 16.83it/s]


(33355474, 33690000, 60393856, 27038382, 1.8025640249332147)

In [132]:
backtest_model_with_cap(with_position_mnl, 100, history_rounds)

100%|██████████| 3369/3369 [03:26<00:00, 16.34it/s]


(3362857, 3369000, 6586100, 3223243, 1.9567358266547936)

In [133]:
backtest_model_with_cap(with_position_mnl, 10, history_rounds)

100%|██████████| 3369/3369 [03:28<00:00, 16.18it/s]


(336868, 336900, 588040, 251172, 1.7455387355298309)

In [134]:
backtest_model_with_cap(with_position_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:28<00:00, 16.14it/s]


(33689, 33690, 58804, 25115, 1.7454734342534877)

In [164]:
# next, we train a model where pirates share one coeff for favorites and one coeff for allergies
split_fa_specification = OrderedDict()
split_fa_names = OrderedDict()
split_fa_specification["intercept"] = pirates_except_the_first
split_fa_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
split_fa_specification["pfa"] = [pirates_including_the_first]
split_fa_names["pfa"] = ['PFA']
split_fa_specification["nfa"] = [pirates_including_the_first]
split_fa_names["nfa"] = ['NFA']
split_fa_specification["position"] = [pirates_including_the_first]
split_fa_names["position"] = ['position']

split_fa_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=split_fa_specification,
model_type='MNL',
names=split_fa_names)
split_fa_mnl.fit_mle(np.zeros(22))
split_fa_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.36 seconds.
Final log-likelihood: -18,142.6405


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,823
Method:                                MLE   Df Model:                           22
Date:                     Wed, 04 Jan 2023   Pseudo R-squ.:                   0.223
Time:                             18:59:28   Pseudo R-bar-squ.:               0.222
AIC:                            36,329.281   Log-Likelihood:            -18,142.641
BIC:                            36,499.381   LL-Null:                   -23,352.129
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.1866      0.058    -20.518      0.000      -1.300      -1.073
ASC_3_Orvinn          -2.1865      0.068    -32.337      0.000      -2.319      -2.054
ASC_4_Lucky           -0.5921      0.055    -10.840      0.000      -0.699      -0.485
ASC_5_Edmund          -0.7312      0.055    -13.277      0.000      -0.839      -0.623
ASC_6_Peg Leg         -1.4406      0.065    -22.283      0.000      -1.567      -1.314
ASC_7_Bonnie          -1.1051      0.056    -19.577      0.000      -1.216      -0.994
ASC_8_Puffo           -1.6565      0.064    -25.919      0.000      -1.782      -1.531
ASC_9_Stuff           -2.4872      0.084    -29.661      0.000      -2.652      -2.323
ASC_10_Squire         -2.2411      0.081    -27.696      0.000      -2.400      -2.083
ASC_11_Crossblades    -2.0420      0.068    -30.065      0.000      -2.175      -1.909
ASC_12_Stripey        -1.2472      0.056    -22.210      0.000      -1.357      -1.137
ASC_13_Ned            -0.6811      0.056    -12.134      0.000      -0.791      -0.571
ASC_14_Fairfax        -1.4607      0.059    -24.900      0.000      -1.576      -1.346
ASC_15_Gooblah         0.9023      0.052     17.379      0.000       0.801       1.004
ASC_16_Franchisco     -0.5020      0.052     -9.685      0.000      -0.604      -0.400
ASC_17_Federismo      -0.5001      0.052     -9.645      0.000      -0.602      -0.398
ASC_18_Blackbeard     -1.2987      0.060    -21.677      0.000      -1.416      -1.181
ASC_19_Buck            0.4531      0.053      8.526      0.000       0.349       0.557
ASC_20_Tailhook       -0.4847      0.054     -8.922      0.000      -0.591      -0.378
PFA                    0.2134      0.009     24.921      0.000       0.197       0.230
NFA                    0.4185      0.012     34.918      0.000       0.395       0.442
position               0.2510      0.008     31.597      0.000       0.235       0.267
======================================================================================
"""

In [146]:
backtest_model_with_cap(split_fa_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:25<00:00, 16.41it/s]


(33682, 33690, 96461, 62779, 2.863431285247848)

In [147]:
backtest_model_with_cap(split_fa_mnl, 10000, history_rounds)

100%|██████████| 3369/3369 [03:07<00:00, 17.99it/s]


(326782178, 336900000, 649728421, 322946243, 1.9585819026417335)

In [ ]:
# OMFG splitting favorite count and allergy count is absolutely necessary!

In [168]:
# next step is to split the coefficients for positional (dis-)advantages
split_position_specification = OrderedDict()
split_position_names = OrderedDict()
split_position_specification["intercept"] = pirates_except_the_first
split_position_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_the_first]
# we use the same FA coefficient and same opening implied winrate coefficient for all pirates
split_position_specification["pfa"] = [pirates_including_the_first]
split_position_names["pfa"] = ['PFA']
split_position_specification["nfa"] = [pirates_including_the_first]
split_position_names["nfa"] = ['NFA']
# split_position_specification["is_pos1"] = [pirates_including_the_first]
# split_position_names["is_pos1"] = ['is_pos1']
split_position_specification["is_pos2"] = [pirates_including_the_first]
split_position_names["is_pos2"] = ['is_pos2']
split_position_specification["is_pos3"] = [pirates_including_the_first]
split_position_names["is_pos3"] = ['is_pos3']
split_position_specification["is_pos4"] = [pirates_including_the_first]
split_position_names["is_pos4"] = ['is_pos4']

split_position_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=split_position_specification,
model_type='MNL',
names=split_position_names)
split_position_mnl.fit_mle(np.zeros(24))
split_position_mnl.get_statsmodels_summary()

Log-likelihood at zero: -23,352.1285
Initial Log-likelihood: -23,352.1285


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.37 seconds.
Final log-likelihood: -18,138.9254


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               16,845
Model:             Multinomial Logit Model   Df Residuals:                   16,821
Method:                                MLE   Df Model:                           24
Date:                     Wed, 04 Jan 2023   Pseudo R-squ.:                   0.223
Time:                             19:04:15   Pseudo R-bar-squ.:               0.222
AIC:                            36,325.851   Log-Likelihood:            -18,138.925
BIC:                            36,511.414   LL-Null:                   -23,352.129
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
ASC_2_Sproggie        -1.1877      0.058    -20.528      0.000      -1.301      -1.074
ASC_3_Orvinn          -2.1886      0.068    -32.358      0.000      -2.321      -2.056
ASC_4_Lucky           -0.5932      0.055    -10.860      0.000      -0.700      -0.486
ASC_5_Edmund          -0.7330      0.055    -13.306      0.000      -0.841      -0.625
ASC_6_Peg Leg         -1.4415      0.065    -22.291      0.000      -1.568      -1.315
ASC_7_Bonnie          -1.1063      0.056    -19.592      0.000      -1.217      -0.996
ASC_8_Puffo           -1.6580      0.064    -25.933      0.000      -1.783      -1.533
ASC_9_Stuff           -2.4890      0.084    -29.677      0.000      -2.653      -2.325
ASC_10_Squire         -2.2447      0.081    -27.729      0.000      -2.403      -2.086
ASC_11_Crossblades    -2.0439      0.068    -30.086      0.000      -2.177      -1.911
ASC_12_Stripey        -1.2485      0.056    -22.226      0.000      -1.359      -1.138
ASC_13_Ned            -0.6826      0.056    -12.160      0.000      -0.793      -0.573
ASC_14_Fairfax        -1.4623      0.059    -24.919      0.000      -1.577      -1.347
ASC_15_Gooblah         0.9004      0.052     17.341      0.000       0.799       1.002
ASC_16_Franchisco     -0.5027      0.052     -9.697      0.000      -0.604      -0.401
ASC_17_Federismo      -0.5015      0.052     -9.669      0.000      -0.603      -0.400
ASC_18_Blackbeard     -1.2998      0.060    -21.691      0.000      -1.417      -1.182
ASC_19_Buck            0.4517      0.053      8.499      0.000       0.348       0.556
ASC_20_Tailhook       -0.4859      0.054     -8.943      0.000      -0.592      -0.379
PFA                    0.2135      0.009     24.925      0.000       0.197       0.230
NFA                    0.4186      0.012     34.926      0.000       0.395       0.442
is_pos2                0.1823      0.027      6.751      0.000       0.129       0.235
is_pos3                0.4633      0.026     17.850      0.000       0.412       0.514
is_pos4                0.7334      0.025     29.093      0.000       0.684       0.783
======================================================================================
"""

In [ ]:
# positional advantage is non-linear,
# so splitting it into 3 coefficients makes sense

In [169]:
backtest_model_with_cap(split_position_mnl, 1, history_rounds)

100%|██████████| 3369/3369 [03:25<00:00, 16.39it/s]


(33683, 33690, 83862, 50179, 2.489433066191748)

In [ ]:
# even though this is theoretically a better model than the previous one
# (it takes into account of non-linearity of positional advantage)
# the backtest ROI result is worse (dropped from 2.863 to 2.489)
# it probably takes an eternity for a model to converge to its true ROI
# given how long-tail the gambit bets are!

In [170]:
backtest_model_with_cap(split_position_mnl, 10000, history_rounds)

100%|██████████| 3369/3369 [03:10<00:00, 17.73it/s]


(326868956, 336900000, 659088561, 332219605, 1.9861074651231818)